# Masrk RCNN Train and Inference

The following notebook uses the Detectron2 framework to train and perform
inference using a MaskRCNN model. This is one of the base models used largely
in object detection problems.

In [ ]:
# if detectron2 is not installed, uncomment this line
# !pip install 'git+https://github.com/facebookresearch/detectron2.git' 'fvcore==0.1.5.post20221221' tabulate

In [ ]:
# Original Src: https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=HUjkwRsOn1O0

import os
import sys

# import some common libraries
import numpy as np
import json
import cv2
import pdb
import time
import random
from pathlib import Path

# if path is not found, uncomment this line and replace
# with your username
# sys.path.append('/home/jacaraba/.local/lib/python3.10/site-packages')

# import detectron2 libraries
#from detectron2.utils.logger import setup_logger

#setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

In [ ]:
# directory where main data is located
base_dir = '/explore/nobackup/people/jacaraba/development/forest-health-gliht/data'
output_dir = '/explore/nobackup/people/jacaraba/development/forest-health-gliht/output'

## Train on my Data

In [ ]:
flight = 'OR_20190630_Three_Creek'
train_dir = os.path.join(base_dir, 'datasets', 'OR_20190630_Three_Creek', 'train')
test_dir = os.path.join(base_dir, 'datasets', 'OR_20190630_Three_Creek', 'test')
val_dir = os.path.join(base_dir, 'datasets', 'OR_20190630_Three_Creek', 'val')

In [ ]:
# coco formatted dataset
train_json = os.path.join(base_dir, 'OR_20190630_Three_Creek_train.json')
test_json = os.path.join(base_dir, 'OR_20190630_Three_Creek_test.json')
val_json = os.path.join(base_dir, 'OR_20190630_Three_Creek_val.json')

In [ ]:
# register the datasets
register_coco_instances(f'{flight}_train', {}, train_json, train_dir)

In [ ]:
# register the datasets
register_coco_instances(f'{flight}_test', {}, test_json, test_dir)

In [ ]:
# register the datasets
register_coco_instances(f'{flight}_val', {}, val_json, val_dir)

## Fine-tune a COCO-pretrained R50-FPN Mask R-CNN model on the dataset.

In [ ]:
# It takes ~6 minutes to train 300 iterations on Colab's K80 GPU, or ~2 minutes on a P100 GPU
cfg = get_cfg()
#cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg._BASE_ = "../Base-RCNN-FPN.yaml"
cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"
cfg.MODEL.MASK_ON = True
cfg.MODEL.RESNETS.DEPTH = 50
cfg.SOLVER.STEPS = (210000, 250000)
cfg.SOLVER.MAX_ITER = 270000

In [ ]:
cfg.DATASETS.TRAIN = (f'{flight}_val',)
cfg.DATASETS.VAL = (f'{flight}_test',)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4 # can increase with better computer

# Let training initialize from model zoo
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2 # dataset-dependent, try out 4-10, good computer start high (32)
cfg.SOLVER.BASE_LR = 0.000025  # pick a good LR
cfg.SOLVER.MAX_ITER = 10  # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512) #lowering can help for prod
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class
cfg.OUTPUT_DIR = output_dir
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS=False # Src: https://github.com/facebookresearch/detectron2/issues/819

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
final_model_path = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
if os.path.isfile(final_model_path):
    os.remove(final_model_path)
    
if not os.path.isfile(final_model_path): #don't rerun training unless I cleared the old one
  trainer = DefaultTrainer(cfg) 
  trainer.resume_or_load(resume=False)
  trainer.train()

## Inference & evaluation using the trained model

In [ ]:
# First, let's create a predictor using the model we just trained
# Inference should use the config with parameters that are used in training
# Changes for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75   # set a custom testing threshold (from 0.63)
predictor = DefaultPredictor(cfg)

In [ ]:
metadata = MetadataCatalog.get(f'{flight}_test')
dataset_dicts = DatasetCatalog.get(f'{flight}_test')
cfg.DATASETS.TEST = (f'{flight}_test',)

In [ ]:
input_prediction = '../data/datasets/OR_20190630_Three_Creek/test/OR_20190630_Three_Creek_c3r2_c2r11_00.png'

In [ ]:
# example inference
im = cv2.imread(input_prediction)
outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
v = Visualizer(im[:, :, ::-1],
             metadata=metadata, 
             scale=1, 
)
v.draw_instance_predictions(outputs["instances"].to("cpu"))

In [ ]:
v = v.get_output()
cv2.imwrite(f"prediction_{Path(input_prediction).stem}.png", v.get_image()[:, :, ::-1]) 